In [1]:
## 인프라_도로 시군구 카운트

def make_infra_file(fnm, write_fnm = None):
    print('write file start', fnm)
    import _config
    import _codes
    import pandas as pd
    import math
    import _util

    fnm_path = _config.infra_facility_read_path + '\\' + fnm + '.xlsx'
    if write_fnm is None: write_fnm_path = _config.infra_write_path + '\\' + fnm
    else: write_fnm_path = _config.infra_write_path + '\\' + write_fnm
    write_fnm_path = write_fnm_path + '.xlsx'

    df = pd.read_excel(fnm_path)
    # print(df.columns)

    #dataframe화 하기
    new_list = []
    for idx, row in df.iterrows():
        #맨 마지막줄 멈춤
        if idx == len(df) - 1: break
        #앞에 쓸모 없는 줄 넘김
        if idx <= 5: continue

        sido_nm = row['Unnamed: 8']
        sido_cd = _util.sido_cd_map.get(sido_nm, None)

        cpt_dt = row['Unnamed: 11']
        if not math.isnan(cpt_dt): 
            cpt_dt = str(cpt_dt)
            year = int(cpt_dt[0:4])
        else: year = 2000
    
        sido_sgg_cd_map = {}
        if year <= 2023: sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2023, 2)
            # year = 2023

        sgg_nm = row['Unnamed: 9']
        sido = sido_sgg_cd_map.get(sido_cd, {})
        sido = dict(sido)
        sgg_cd = sido.get(sgg_nm, None)
        # if sgg_nm == '전라북도': sgg_cd = '35'

        new_list.append({
            'fac_nm': row['Unnamed: 1']
            , 'fac_sort': row['Unnamed: 2']
            , 'admin_sys1': row['Unnamed: 3']
            , 'admin_sys2': row['Unnamed: 4']
            , 'sido_cd': f'`{sido_cd}'
            , 'sido_nm': sido_nm
            , 'sgg_cd': f'`{sgg_cd}'
            , 'sgg_nm': sgg_nm
            , 'emd_nm': row['Unnamed: 10']
            , 'cpt_dt': f'`{cpt_dt}'
            , 'year': f'`{year}'
            , 'sort': write_fnm
        })

    col_list = []
    for col in df.columns:
        if col.count('fac_sort') > 0: continue
        col_list.append(col)

    df = pd.DataFrame(new_list)
    df.to_excel(write_fnm_path, index=False)
    print('write file end', write_fnm)

# print(len(list_file_nm))
import _config
for idx in range(len(_config.infra_file_nm_list)):
    fnm = _config.infra_file_nm_list[idx]
    if fnm == '전기': continue
    make_infra_file(f'{idx + 1}. 인프라 총조사 결과_시설별 현황({fnm})', fnm)

write file start 1. 인프라 총조사 결과_시설별 현황(도로)
write file end 도로
write file start 2. 인프라 총조사 결과_시설별 현황(철도)
write file end 철도
write file start 3. 인프라 총조사 결과_시설별 현황(항만, 어항)
write file end 항만, 어항
write file start 4. 인프라 총조사 결과_시설별 현황(공항)
write file end 공항
write file start 6. 인프라 총조사 결과_시설별 현황(가스)
write file end 가스
write file start 7. 인프라 총조사 결과_시설별 현황(열공급)
write file end 열공급
write file start 8. 인프라 총조사 결과_시설별 현황(통신)
write file end 통신
write file start 9. 인프라 총조사 결과_시설별 현황(공동구)
write file end 공동구
write file start 10. 인프라 총조사 결과_시설별 현황(송유)
write file end 송유
write file start 11. 인프라 총조사 결과_시설별 현황(하천)
write file end 하천
write file start 12. 인프라 총조사 결과_시설별 현황(저수지)
write file end 저수지
write file start 13. 인프라 총조사 결과_시설별 현황(댐)
write file end 댐
write file start 14. 인프라 총조사 결과_시설별 현황(하수도)
write file end 하수도


In [2]:
import _util
import pandas as pd
import _config

def create_table(columns):
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

        if str(columns[idx]).count('fac_nm') > 0: columns[idx] = columns[idx] + ' varchar(100)'
        if str(columns[idx]).count('fac_sort') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('admin_sys1') > 0: columns[idx] = columns[idx] + ' varchar(20)'
        if str(columns[idx]).count('admin_sys2') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sido_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sido_cd') > 0: columns[idx] = columns[idx] + ' varchar(2)'
        if str(columns[idx]).count('sgg_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sgg_cd') > 0: columns[idx] = columns[idx] + ' varchar(5)'
        if str(columns[idx]).count('emd_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('cpt_dt') > 0: columns[idx] = columns[idx] + ' varchar(8)'
        if str(columns[idx]).count('year') > 0: columns[idx] = columns[idx] + ' varchar(4)'
        if str(columns[idx]) == '"sort"': columns[idx] = columns[idx] + ' varchar(10)'

    sql = f'''
    do $$
    begin 
        if exists (select 1 from pg_tables where tablename = 'infra') then
        drop table infra cascade;
        end if;
        create table if not exists infra(
            {', '.join(columns)}
        );
    end $$;
    '''

    _util.execute_sql(sql)

#insert
def insert_data(df, columns):
    #column setting
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'
    
    #to use dataframe for clause
    df = pd.DataFrame(df)
    data = df.values.tolist()
    for chunk in _util.chunker(data, 1000):
        values = []
        for row in chunk:
            fac_nm, fac_sort, admin_sys1, admin_sys2, sido_cd, sido_nm, sgg_cd, sgg_nm, emd_nm, cpt_dt, year, sort = row
            if sido_cd == '`None' or sido_cd is None: continue

            for idx in range(len(columns)):
                if str(row[idx]).count('`') > 0:
                    val = str(row[idx]).replace('`', '')
                    row[idx] = f'\'{val}\''
                else: row[idx] = f'\'{str(row[idx])}\''
                if row[idx] == 'None' or row[idx] is None: row[idx] = '\'\''

            values.append(f"({', '.join(row)})")
        if len(values) == 0: continue
        
        sql = f'''
            insert into infra(
                {','.join(columns)})
            values {', '.join(values)}
        '''
        _util.execute_sql(sql)

frt = False
for fnm in _config.infra_file_nm_list:
    if fnm == '전기': continue
    df = pd.read_excel(_config.infra_write_path + '\\' + fnm + '.xlsx')
    if not frt: 
        create_table(df.columns)
        frt = not frt
    print('data insert start', fnm)
    insert_data(df, df.columns)
    print('data insert end', fnm)

data insert start 도로
data insert end 도로
data insert start 철도
data insert end 철도
data insert start 항만, 어항
data insert end 항만, 어항
data insert start 공항
data insert end 공항
data insert start 가스
data insert end 가스
data insert start 열공급
data insert end 열공급
data insert start 통신
data insert end 통신
data insert start 공동구
data insert end 공동구
data insert start 송유
data insert end 송유
data insert start 하천
data insert end 하천
data insert start 저수지
data insert end 저수지
data insert start 댐
data insert end 댐
data insert start 하수도
data insert end 하수도


In [21]:
# infra 데이터 가공하기
# 노후년도 15년 단위로 

import _util
import _config
import pandas as pd

# infras = {
#     'road': 'admin_sys1 like \'%도로%\'',
#     'cmm_tunnel': 'admin_sys1 like \'%공동구%\'',
#     'gas': 'fac_sort like \'%가스%\'',
#     'rail': 'admin_sys1 like \'%철도%\'',
#     'cd': 'admin_sys1 like \'%통신구%\'',
#     'heat': 'fac_sort like \'%열수송관%\''
# }

def infra_datafram(year, sort, unit_year):
    sql = f'''
        with gu as (
            select *
            from gujaesi_sgg_{year}_5179 gs            
        )
        , ngu as (
            select *
            from nogujaesi_sgg_{year}_5179 gs            
        )
        , i as (
            select sgg_cd, sgg_nm
            , count(*) filter (where "노후년도" between 0 and {unit_year - 1}) as "{unit_year}년 미만"
            , count(*) filter (where "노후년도" between {unit_year} and {unit_year * 2}) as "{unit_year} ~ {unit_year * 2}년"
            , count(*) filter (where "노후년도" > {unit_year * 2}) as "{unit_year * 2}년 초과"
            , sort
            from (
                select sgg_cd, sgg_nm, sort, '{year}'::integer - i."year"::integer as "노후년도", i."year"
                from infra i
        --		where sort not in ('공항', '댐', '하천', '항만, 어항', '저수지') 
                where sort = '{sort}' and "year" <= '{year}'
            )
            group by sgg_cd, sgg_nm, sort
        )
        select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '{sort}') as sort
        , coalesce(i."{unit_year}년 미만", 0) as "{unit_year}년 미만"
        , coalesce(i."{unit_year} ~ {unit_year * 2}년", 0) as "{unit_year} ~ {unit_year * 2}년"
        , coalesce(i."{unit_year * 2}년 초과", 0) as "{unit_year * 2}년 초과"
        from ngu g
        left join i on i.sgg_cd = g.sgg_cd
        union all 
        select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '{sort}') as sort
        , coalesce(i."{unit_year}년 미만", 0) as "{unit_year}년 미만"
        , coalesce(i."{unit_year} ~ {unit_year * 2}년", 0) as "{unit_year} ~ {unit_year * 2}년"
        , coalesce(i."{unit_year * 2}년 초과", 0) as "{unit_year * 2}년 초과"
        from gu g
        left join i on i.sgg_cd = g.sgg_cd
        ;
    '''

    result = _util.execute_sql(sql)
    if result is None: return

    _list = []
    for row in result:
        sgg_cd, sgg_nm, sort, unit1, unit2, unit3 = row 
        _list.append({
            'sgg_cd': sgg_cd
            , 'sgg_nm': sgg_nm
            , 'sort': sort
            , f'{unit_year} 미만': unit1
            , f'{unit_year} ~ {unit_year * 2}년': unit2
            , f'{unit_year * 2}년 초과': unit3
        })

    df = pd.DataFrame(_list)

    sql = f'''
        do $$
        begin 
            if exists (select 1 from pg_matviews where matviewname = 'v_infra_{sort}_{year}_5179') then
            drop materialized view v_infra_{sort}_{year}_5179;
            end if;
            create materialized view v_infra_{sort}_{year}_5179 as 
            with gu as (
                select *
                from gujaesi_sgg_{year}_5179 gs            
            )
            , ngu as (
                select *
                from nogujaesi_sgg_{year}_5179 gs            
            )
            , i as (
                select sgg_cd, sgg_nm
                , count(*) filter (where "노후년도" between 0 and {unit_year - 1}) as "{unit_year}년 미만"\
                
                , count(*) filter (where "노후년도" between {unit_year} and {unit_year * 2}) as "{unit_year} ~ {unit_year * 2}년"
                , count(*) filter (where "노후년도" > {unit_year * 2}) as "{unit_year * 2}년 초과"
                , sort
                from (
                    select sgg_cd, sgg_nm, sort, '{year}'::integer - i."year"::integer as "노후년도", i."year"
                    from infra i
            --		where sort not in ('공항', '댐', '하천', '항만, 어항', '저수지') 
                    where sort = '{sort}' and "year" <= '{year}'
                )
                group by sgg_cd, sgg_nm, sort
            )
            select row_number() over() as fid, *
            from (
                select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '{sort}') as sort
                , coalesce(i."{unit_year}년 미만", 0) as "{unit_year}년 미만"
                , coalesce(i."{unit_year} ~ {unit_year * 2}년", 0) as "{unit_year} ~ {unit_year * 2}년"
                , coalesce(i."{unit_year * 2}년 초과", 0) as "{unit_year * 2}년 초과"
                , g.geom
                from ngu g
                left join i on i.sgg_cd = g.sgg_cd
                union all 
                select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '{sort}') as sort
                , coalesce(i."{unit_year}년 미만", 0) as "{unit_year}년 미만"
                , coalesce(i."{unit_year} ~ {unit_year * 2}년", 0) as "{unit_year} ~ {unit_year * 2}년"
                , coalesce(i."{unit_year * 2}년 초과", 0) as "{unit_year * 2}년 초과"
                , g.geom
                from gu g
                left join i on i.sgg_cd = g.sgg_cd
            )
            ;
        end $$;
    '''
    _util.execute_sql(sql)

    return df

for year in _config.layer_years:
    # w_cnt = 0
    df_list = []    
    for sort in _config.infra_file_nm_list:
        if sort == '전기': continue
        if sort == '공항': continue
        if sort == '댐': continue
        if sort == '하천': continue
        if sort == '항만, 어항': continue
        if sort == '저수지': continue
        if sort == '하수도': continue
        if sort == '송유': continue
        df_list.append(infra_datafram(sort=sort, unit_year=15, year=year))
        # w_cnt += 1
    df = pd.concat(df_list, ignore_index=True)
    df.to_csv(_config.infra_write_path + '\\' + f'infra_전국_데이터_{year}' + '.csv', encoding=_config.euc_kr, index=False)
    # print(w_cnt)


In [32]:
import _util
import pandas as pd
import _config 
import re

patterns = {
    'geom': re.compile(r'geom'),
}


def makeCsv(table_nm):
    cols = _util.getMatViewCols(table_nm)
    if cols is None: return
    cols = [col[0] for col in cols if not patterns['geom'].search(col[0])]

    sql = f'''
        select {', '.join(cols)}
        from {table_nm}
    '''
    result = _util.execute_sql(sql)

    if result is None: return
    _list = []
    for row in result: 
        print(row)
        _list.append({cols[idx]: row[idx] for idx in len(cols)})
    df = pd.DataFrame(_list)
    _list = None
    df.to_csv(_config.infra_write_path + '\\' + 'infra_2023_v2' + '.csv', encoding=_config.euc_kr, index=False)

table_nm = 'v_infra_2023_5179'
makeCsv(table_nm)
# sql = f'''
#     select 
# '''

# result = _util.execute_sql()



(1, '11010', '종로구', 150, 89, 456, 20, 33, 177, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0)


TypeError: 'int' object is not iterable

In [8]:


cc = f'''
-- 0528

create materialized view v_infra_2023_5179 as
select row_number() over() as fid
, ra.sgg_cd, ra.sgg_nm, ra."15년 미만" as ra_15, ra."15 ~ 30년" as ra_15_30, ra."30년 초과" as ra_30
, ro."15년 미만" as ro_15, ro."15 ~ 30년" as ro_15_30, ro."30년 초과" as ro_30
, ct."15년 미만" as ct_15, ct."15 ~ 30년" as ct_15_30, ct."30년 초과" as ct_30
, ht."15년 미만" as ht_15, ht."15 ~ 30년" as ht_15_30, ht."30년 초과" as ht_30
, cmt."15년 미만" as cmt_15, cmt."15 ~ 30년" as cmt_15_30, cmt."30년 초과" as cmt_30
, ga."15년 미만" as ga_15, ga."15 ~ 30년" as ga_15_30, ga."30년 초과" as ga_30
, ra.geom
from v_infra_철도_2023_5179 ra
left join v_infra_도로_2023_5179 ro on ro.sgg_cd = ra.sgg_cd
left join v_infra_통신_2023_5179 ct on ct.sgg_cd = ra.sgg_cd
left join v_infra_열공급_2023_5179 ht on ht.sgg_cd = ra.sgg_cd
left join v_infra_공동구_2023_5179 cmt on cmt.sgg_cd = ra.sgg_cd
left join v_infra_가스_2023_5179 ga on ga.sgg_cd = ra.sgg_cd
;

with ngu as (
    select *
    from nogujaesi_sgg_2000_5179 gs            
)
, i as (
    select sgg_cd, sgg_nm
    , count(*) filter (where "노후년도" between 0 and 14) as "15 미만"
    , count(*) filter (where "노후년도" between 15 and 30) as "15 ~ 30년"
    , count(*) filter (where "노후년도" > 30) as "30년 초과"
    , sort
    from (
        select sgg_cd, sgg_nm, sort, '2000'::integer - i."year"::integer as "노후년도", i."year"
        from infra i
--		where sort not in ('공항', '댐', '하천', '항만, 어항', '저수지') 
        where sort = '도로'
    )
    group by sgg_cd, sgg_nm, sort
)
select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '도로')
, coalesce(i."15 미만", 0) as "15년 미만"
, coalesce(i."15 ~ 30년", 0) as "15 ~ 30년"
, coalesce(i."30년 초과", 0) as "30년 초과"
from ngu g
left join i on i.sgg_cd = g.sgg_cd

--fac_sort = '가스배관'
--admin_sys1 = '공동구'
--admin_sys1 = '도로'
--admin_sys1 like '%철도%'
--admin_sys1 = '통신구'
--fac_sort = '열수송관'

with ngu as (
	select *
	from nogujaesi_sgg_2023_5179 gs            
)
, i as (
	select sgg_cd, sgg_nm
	, count(*) filter (where "노후년도" between 0 and 14) as "15년 미만"
	, count(*) filter (where "노후년도" between 15 and 30) as "15 ~ 30년"
	, count(*) filter (where "노후년도" >= 30) as "30년 초과"
	, sort
	from (
		select sgg_cd, sgg_nm, sort, '2023'::integer - i."year"::integer as "노후년도", i."year"
		from infra i
--		where sort not in ('공항', '댐', '하천', '항만, 어항', '저수지') 
		where sort = '통신'
	)
	group by sgg_cd, sgg_nm, sort
)
select g.sgg_cd, g.sgg_nm, coalesce (i.sort, '통신')
, coalesce(i."15년 미만", 0) as "15년 미만"
, coalesce(i."15 ~ 30년", 0) as "15 ~ 30년"
, coalesce(i."30년 초과", 0) as "30년 초과"
--select count(*)
from ngu g
left join i on i.sgg_cd = g.sgg_cd
--where sort is null
--, "노후년도"
;


with gu as (
	select *
	from gujaesi_sgg_2023_5179 gs            
)
, i as (
	select *
	from infra
	where sort not in ('공항', '댐', '하천', '항만, 어항')
)
select sgg_cd, sgg_nm, sort
--, fac_sort, admin_sys1
, count(*) filter (where "노후년도" between 0 and 14) as "15년 미만"
, count(*) filter (where "노후년도" between 15 and 30) as "15 ~ 30년"
, count(*) filter (where "노후년도" >= 30) as "30년 초과"
from (
	select g.sgg_cd, g.sgg_nm
	, count(i.fac_nm) as fac_cnt
	, date_part('year', now())::integer - i."year"::integer as "노후년도"
	, i.sort
	from gu g
	left join i on i.sgg_cd = g.sgg_cd
	group by g.sgg_cd, g.sgg_nm, i."year", i.sort
	order by g.sgg_cd, date_part('year', now())::integer - i."year"::integer
)
group by sgg_cd, sgg_nm, sort
order by sort, sgg_cd
--, "노후년도"
;
'''